In [ ]:
%env MODEL_DIR=/content/Cats
%env OUT_DIR=/content/drive/MyDrive/nn/cats3

# Install Dependencies

In [ ]:
%env DARKNET_VERSION=5.1.82
!if [[ ! -d /content/darknet_tflite ]]; then git clone --depth 1 --branch main https://github.com/sventschui/darknet-tflite.git /content/darknet_tflite; fi
!/content/darknet_tflite/scripts/install_dependencies.sh

In [ ]:
!cd /content/Cats && darknet detector calc_anchors /content/Cats/Cats.data -num_of_clusters 6 -width 384 -height 288


# Unpack dataset

In [ ]:
!/content/darknet_tflite/scripts/unpack_cats.sh

# Train model

In [ ]:
# Mount GDrive, only possible in cloud colab notebooks
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%bash

set -e

rm -rf /content/drive/MyDrive/nn/cats2
mkdir -p /content/drive/MyDrive/nn/cats2
cp "${MODEL_DIR}/Cats.cfg" "${MODEL_DIR}/Cats.data" /content/drive/MyDrive/nn/cats2

cd /content/drive/MyDrive/nn/cats2

darknet detector train \
  "${MODEL_DIR}/Cats.data" \
  "${MODEL_DIR}/Cats.cfg" \
  -map \
  -dont_show #\
  # Cat_best.weights \
  # -clear

cp "${MODEL_DIR}"/*.weights /content/drive/MyDrive/nn/cats2

# Export ONNX model

In [ ]:
%%bash
set -e
cd "${MODEL_DIR}"
darknet_onnx_export -noboxes Cats.cfg Cats_best.weights Cats.names

cp "${MODEL_DIR}"/*.onnx "${OUT_DIR}"

# Convert to tflite

In [ ]:
%cd /content/darknet_tflite
import cv2
import glob
import numpy as np
import os

import functions
functions.reload()

DARKNET_CONFIG_PATH = f"{os.environ.get("MODEL_DIR")}/Cats.cfg"
IMAGES_PATH = f"{os.environ.get("MODEL_DIR")}/set_01"

net, layers = functions.parse_darknet_cfg(DARKNET_CONFIG_PATH)

files = glob.glob(f"{IMAGES_PATH}/*.jpg")
images = []
for idx, file in enumerate(files):
    img = cv2.imread(file)
    img = cv2.resize(img, dsize=(net["width"], net["height"]))
    img = img.astype(np.float32) / 255.0  # convert to 0.0 - 1.0 scale
    images.append(img)

np.save(file=f"{IMAGES_PATH}/calibdata.npy", arr=np.stack(images, axis=0))

In [ ]:
%%bash

set -e

cd "${MODEL_DIR}"

onnx2tf -i Cats.onnx \
    -oiqt \
    -o "${MODEL_DIR}"  \
    -cind "frame" "set_01/calibdata.npy" "[[[[0,0,0]]]]" "[[[[1,1,1]]]]"

cp "${MODEL_DIR}"/*.tflite "${OUT_DIR}"

# Compile for edgetpu

In [ ]:
%%bash

set -e

cd "${MODEL_DIR}"

edgetpu_compiler "Cats_full_integer_quant.tflite"

cp "${MODEL_DIR}/Cats_full_integer_quant_edgetpu.tflite" "${OUT_DIR}"